In [1]:
import pandas as pd
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()  

True

In [2]:
MOVIES_DF = pd.read_csv("../data/cleaned/final_df.csv")

In [3]:
MOVIES_DF['date'] = MOVIES_DF['release_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [4]:
ids = MOVIES_DF.sort_values(['date'], ascending=False)['imdb_id'].tolist()
print(f"Found {len(ids)} movie IDs to fetch.")

# Create batches of 1000 ids
batch_size = 1000
batches = [ids[i:i + batch_size] for i in range(0, len(ids), batch_size)]
print(f"Created {len(batches)} batches of up to {batch_size} IDs each.")


Found 10067 movie IDs to fetch.
Created 11 batches of up to 1000 IDs each.


In [5]:
MOVIES_DF[MOVIES_DF.date > '1990-01-01']

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,...,vote_average,vote_count,origin_country,spoken_languages,genre_ids,genre_names,production_company_ids,production_company_names,belongs_to_collection,gross_per_theater
0,PARA,2016-03-10,10 Cloverfield Lane,Paramount Pi…,11414,0.32,-0.12,120.0,95.0,72082999,...,7.000,8351.0,US,English,"53, 878, 18, 27","Thriller, Science Fiction, Drama, Horror",11461,Bad Robot,NaN,95.116667
1,Private,2006-08-17,10th & Wolf,ThinkFilm,1791,0.00,0.00,6.0,299.0,49783,...,5.856,108.0,US,English,"28, 80, 18, 9648, 53","Action, Crime, Drama, Mystery, Thriller",41427,Suzanne DeLaurentiis Productions,NaN,298.500000
2,6758,2009-03-03,12,Sony Picture…,344,0.00,0.00,5.0,69.0,119587,...,5.600,50.0,US,English,18,Drama,NaN,NaN,NaN,68.800000
3,6758,2009-03-03,12,Sony Picture…,344,0.00,0.00,5.0,69.0,119587,...,5.000,57.0,US,English,35,Comedy,NaN,NaN,NaN,68.800000
4,DIS,2009-03-26,12 Rounds,20th Century…,4832,0.00,0.98,29.0,167.0,12187944,...,5.904,819.0,US,English,"28, 53, 80","Action, Thriller, Crime","1557, 17887, 2890, 10339","The Mark Gordon Company, Midnight Sun Pictures...",12 Rounds Collection,166.620690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10062,SONY,2011-07-07,Zookeeper,Sony Pictures,13006,-0.32,-0.57,94.0,138.0,80360866,...,5.428,1730.0,US,English,"35, 10749, 10751","Comedy, Romance, Family","8000, 46222, 878, 5, 21","Broken Road Productions, Hey Eddie, Happy Madi...",NaN,138.361702
10063,PARA,2001-09-27,Zoolander,Paramount Pi…,1430000,0.49,-0.38,2285.0,626.0,39272000,...,6.207,4697.0,US,English,35,Comedy,"4, 79, 268652, 172, 258, 2932","Paramount Pictures, Village Roadshow Pictures,...",Zoolander Collection,625.820569
10064,PARA,2016-02-11,Zoolander 2,Paramount Pi…,2649,-0.21,-0.74,60.0,44.0,28848693,...,4.921,2306.0,US,English,"35, 12, 28","Comedy, Adventure, Action","4, 2932, 258","Paramount Pictures, Red Hour, Scott Rudin Prod...",Zoolander Collection,44.150000
10065,SONY,2006-08-10,Zoom,Sony Pictures,94312,0.00,0.00,303.0,311.0,11406340,...,5.306,624.0,US,English,"10751, 35, 12, 28, 878","Family, Comedy, Adventure, Action, Science Fic...","497, 598, 12626","Revolution Studios, Team Todd, Boxing Cat Films",NaN,311.260726


In [6]:
from utils import query_omdb, get_movies_from_ids

## Extract Movie Details from OMDB API



In [7]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")

Note that there are a bunch of different endpoints for different sorts of info, meaning we could query by:

View API Reference here: [OMDB API Reference](www.omdbapi.com)

In [8]:
example_movie = query_omdb("tt0111161", OMDB_API_KEY)

In [16]:
pd.DataFrame([example_movie])

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Rating_InternetMovieDatabase,Rating_RottenTomatoes,Rating_Metacritic
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton",A banker convicted of uxoricide forms a friend...,...,tt0111161,movie,N/A,"$28,767,189",N/A,N/A,True,9.3/10,89%,82/100


### Collecting Data batches

In [10]:
collect = True
all_movies = []

In [17]:
for i in range(len(batches)):
    all_movies = []
    batch = batches[i]
    movies = get_movies_from_ids(OMDB_API_KEY, batches[i], limit=1000, verbose=True)
    print(f"Retrieved {len(batch)} movies from batch {i}")
    print(f"Total movies so far: {len(all_movies)}")

    pd.DataFrame(movies).to_csv(f"omdbmovies_batch_{i}.csv", index=False)
    

Requesting movie 0...
Requesting movie 1...
Requesting movie 2...
Requesting movie 3...
Requesting movie 4...
Requesting movie 5...
Requesting movie 6...
Requesting movie 7...
Requesting movie 8...
Requesting movie 9...
Requesting movie 10...
Requesting movie 11...
Requesting movie 12...
Requesting movie 13...
Requesting movie 14...
Requesting movie 15...
Requesting movie 16...
Requesting movie 17...
Requesting movie 18...
Requesting movie 19...
Requesting movie 20...
Requesting movie 21...
Requesting movie 22...
Requesting movie 23...
Requesting movie 24...
Requesting movie 25...
Requesting movie 26...
Requesting movie 27...
Requesting movie 28...
Requesting movie 29...
Requesting movie 30...
Requesting movie 31...
Requesting movie 32...
Requesting movie 33...
Requesting movie 34...
Requesting movie 35...
Requesting movie 36...
Requesting movie 37...
Requesting movie 38...
Requesting movie 39...
Requesting movie 40...
Requesting movie 41...
Requesting movie 42...
Requesting movie 43..

In [18]:
import glob

In [19]:
csv_files = glob.glob("omdbmovies_batch_*.csv")

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # Remove rows where Title is null, empty, or just commas
    df = df[df["Title"].notna()]          # drop NaNs
    df = df[df["Title"].str.strip() != ""]  # drop empty strings
    dfs.append(df)

# Merge everything
omdb_merged = pd.concat(dfs, ignore_index=True)

In [20]:
omdb_merged

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Website,Response,Rating_InternetMovieDatabase,Rating_RottenTomatoes,Rating_Metacritic,Season,Episode,seriesID,Error,totalSeasons
0,Men in Black,1934.0,Passed,28 Sep 1934,19 min,"Comedy, Short",Ray McCarey,Felix Adler,"Moe Howard, Larry Fine, Curly Howard",The stooges become doctors at a large hospital...,...,NaN,True,7.7/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Men in Black,1997.0,PG-13,02 Jul 1997,98 min,"Action, Adventure, Comedy",Barry Sonnenfeld,"Lowell Cunningham, Ed Solomon","Tommy Lee Jones, Will Smith, Linda Fiorentino","James, an NYC cop, is hired by Agent K of a se...",...,NaN,True,7.3/10,91%,72/100,NaN,NaN,NaN,NaN,NaN
2,Face/Off,1997.0,R,27 Jun 1997,138 min,"Action, Crime, Sci-Fi",John Woo,"Mike Werb, Michael Colleary","John Travolta, Nicolas Cage, Joan Allen","To foil a terrorist plot, an FBI agent assumes...",...,NaN,True,7.3/10,93%,82/100,NaN,NaN,NaN,NaN,NaN
3,Head Above Water,2017.0,NaN,05 Oct 2017,11 min,"Short, Drama, Family",Eric Shahinian,"Maya Huang, Eric Shahinian","Anne-Marie Agbodji, Rose Hanish, Alecia Hurst",A devoted husband is forced to confront his do...,...,NaN,True,7.9/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Head Above Water,1996.0,PG-13,25 Jun 1997,92 min,"Comedy, Thriller",Jim Wilson,"Eirik Ildahl, Geir Eriksen, Theresa Marie","Harvey Keitel, Cameron Diaz, Craig Sheffer","A prominent judge, his wife, her ex-lover, and...",...,NaN,True,6.0/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,Contact,2017.0,NaN,10 Jul 2017,8 min,"Animation, Short, Drama",Katy Wang,"Katy Wang, Gabriel de Bruin",NaN,"Stranded on a distant planet, a lonely astrona...",...,NaN,True,5.8/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8477,Contact,2025.0,NaN,19 Sep 2025,NaN,"Short, Romance",NaN,NaN,NaN,In a heartfelt night of vulnerability and laug...,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8478,Contact,1933.0,NaN,19 Sep 2011,28 min,"Documentary, Short",Paul Rotha,NaN,NaN,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8479,Wild America,1997.0,PG,02 Jul 1997,106 min,"Action, Adventure, Comedy",William Dear,David Michael Wieger,"Jonathan Taylor Thomas, Devon Sawa, Scott Bair...",Three brothers who are obsessed with animals a...,...,NaN,True,6.4/10,30%,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
final_df = pd.read_csv("../data/cleaned/final_df.csv")
final_df.columns

Index(['ticker', 'date', 'title', 'distributor', 'gross', 'percent_yd',
       'percent_lw', 'theaters', 'per_theater', 'total_gross',
       'days_in_release', 'parent company', 'release_date', 'year',
       'title_key', 'tmdb_id', 'popularity', 'weekday', 'release_month',
       'release_weekday', 'is_weekend', 'imdb_id', 'original_language',
       'status', 'budget', 'revenue', 'adult', 'overview', 'runtime',
       'vote_average', 'vote_count', 'origin_country', 'spoken_languages',
       'genre_ids', 'genre_names', 'production_company_ids',
       'production_company_names', 'belongs_to_collection',
       'gross_per_theater'],
      dtype='object')

In [24]:
omdb_merged.rename(columns={"imdbID": "imdb_id"}, inplace=True)

In [26]:
final_with_omdb = final_df.merge(omdb_merged, on="imdb_id", how="left")

5725